<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [66]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [67]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [68]:
def x1(x, v):
  return math.sin(math.log(abs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(v[0] / v[2]) - 3**(x[2]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] / v[0]

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [69]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [70]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_0

,x1(0),x2(0),x3(0),v1(0),v2(0),v3(0),y1(0),y2(0),y3(0)
0,0.979192,0.693342,0.063997,0.963676,-54.520348,1.171676,1.207528,0.959897,1.171676
1,-0.016469,0.985913,-0.876941,11.737576,33.038467,0.834694,-0.000181,0.860867,0.834694
2,-0.286341,-0.621831,-0.897366,0.207562,87.796840,0.654753,-0.936143,0.780361,0.654753
3,0.358505,-0.539309,0.550901,986.154945,45.758029,1.245891,0.000296,0.973489,1.245891
4,-0.084503,-0.999441,-0.918911,16.872739,46.298477,0.655107,-0.001456,0.780541,0.655107
...,...,...,...,...,...,...,...,...,...
95,-0.171005,-0.557088,-0.976701,2.340292,-43.910103,0.066127,-0.040484,0.257059,0.066127
96,0.980792,0.882740,0.866777,-0.529938,-80.591627,1.042199,-1.950852,0.929255,1.042199
97,-0.973664,-0.416506,-0.875076,-2.354448,3.395401,-0.465643,0.632286,0.670073,-0.465643
98,0.941083,0.765912,-0.662985,2.201295,1.836972,0.848201,0.473886,0.866079,0.848201


In [71]:
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_1

,x1(1),x2(1),x3(1),v1(1),v2(1),v3(1),y1(1),y2(1),y3(1)
0,-0.589411,-0.698272,0.004210,-2.551372,-74.025025,2.086252,0.212247,0.932775,2.086252
1,0.929686,-0.851138,0.345779,-0.062823,66.346062,-1.248931,-15.466287,0.973985,-1.248931
2,0.961857,0.868315,-0.045032,1.866361,45.830697,5.378301,0.542415,0.886766,5.378301
3,0.029673,-0.754452,-0.913969,-2.614657,48.926436,1.419027,-0.002251,0.994236,1.419027
4,0.979929,-0.468495,0.346637,5.618649,34.664774,3.929742,0.252236,0.842051,3.929742
...,...,...,...,...,...,...,...,...,...
95,0.624661,-0.783657,0.687843,20.780845,18.779322,-0.216916,0.026837,0.463917,-0.216916
96,-0.936173,0.055157,0.007311,4.315472,3.430538,3.170586,-0.893727,0.170263,3.170586
97,-0.045850,-0.372497,0.032599,0.536612,90.001038,0.112769,-0.029977,0.335455,0.112769
98,-0.860497,0.996610,-0.886988,-0.132628,-0.358848,14.621602,6.028724,0.940712,14.621602


In [72]:
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_2

,x1(2),x2(2),x3(2),v1(2),v2(2),v3(2),y1(2),y2(2),y3(2)
0,0.067404,-0.993426,0.581801,-5.623967,79.730013,3.181201,-0.003122,0.198991,3.181201
1,-0.423611,-0.685583,-0.987381,1.847755,-6.183086,1.194217,-0.180209,0.964328,1.194217
2,0.842919,0.991140,-0.556428,8.887412,67.187114,6.759771,0.087465,0.677309,6.759771
3,0.869813,-0.253649,-0.080431,7.971767,-9.855572,1.393814,0.202054,0.992159,1.393814
4,-0.944965,-0.291640,-0.646198,0.016796,15.850719,4.761601,-101.271304,0.999394,4.761601
...,...,...,...,...,...,...,...,...,...
95,-0.805366,-0.997901,-0.228731,2.099056,46.217891,6.464269,-0.344685,0.424376,6.464269
96,-0.405429,-0.549249,-0.521788,-0.655296,44.983700,0.795798,0.531558,0.845235,0.795798
97,-0.011657,-0.964162,0.653152,15.601070,-85.756223,1.592194,-0.000082,0.999886,1.592194
98,0.717946,0.034673,-0.376920,464142.004348,55.914549,15.182905,0.000007,0.708000,15.182905


##1. Линейная форма функции пересчёта

###Для функции $x_1[t+1]$

In [73]:
#x3, v1, v2
vector_ones = [1] * elem
x1_linear = model_0[['x3(0)', 'v1(0)', 'v2(0)']]
x1_linear['x1(1)'] = model_1['x1(1)']
x1_linear.insert(0, "optional_col", vector_ones)
x1_linear

<ipython-input-73-08999791caa7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linear['x1(1)'] = model_1['x1(1)']


,optional_col,x3(0),v1(0),v2(0),x1(1)
0,1,0.063997,0.963676,-54.520348,-0.589411
1,1,-0.876941,11.737576,33.038467,0.929686
2,1,-0.897366,0.207562,87.796840,0.961857
3,1,0.550901,986.154945,45.758029,0.029673
4,1,-0.918911,16.872739,46.298477,0.979929
...,...,...,...,...,...
95,1,-0.976701,2.340292,-43.910103,0.624661
96,1,0.866777,-0.529938,-80.591627,-0.936173
97,1,-0.875076,-2.354448,3.395401,-0.045850
98,1,-0.662985,2.201295,1.836972,-0.860497


In [95]:
A1_linear = np.array(x1_linear)
#A1_psev = BenIsrael(A1_linear[:, 0:(len(A1_linear[0]) - 1)])
X1_estim_linear = solvingSLAE(A1_linear[:, 0:(len(A1_linear[0]) - 1)], np.matrix(A1_linear[:, len(A1_linear[0]) - 1]).T)
print(pd.DataFrame(X1_estim_linear, columns=['X1'], index=['a1', 'c13', 'b11', 'b12']))

           X1
a1   0.003790
c13 -0.421353
b11 -0.000178
b12 -0.000230


###Для функции $x_2[t+1]$

In [86]:
#x2, x3, v1
x2_linear = model_0[['x2(0)', 'x3(0)', 'v1(0)']]
x2_linear['x2(1)'] = model_1['x2(1)']
x2_linear.insert(0, "optional_col", vector_ones)
x2_linear

<ipython-input-86-b7e7da0cd146>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2_linear['x2(1)'] = model_1['x2(1)']


,optional_col,x2(0),x3(0),v1(0),x2(1)
0,1,0.693342,0.063997,0.963676,-0.698272
1,1,0.985913,-0.876941,11.737576,-0.851138
2,1,-0.621831,-0.897366,0.207562,0.868315
3,1,-0.539309,0.550901,986.154945,-0.754452
4,1,-0.999441,-0.918911,16.872739,-0.468495
...,...,...,...,...,...
95,1,-0.557088,-0.976701,2.340292,-0.783657
96,1,0.882740,0.866777,-0.529938,0.055157
97,1,-0.416506,-0.875076,-2.354448,-0.372497
98,1,0.765912,-0.662985,2.201295,0.996610


In [96]:
A2_linear = np.array(x2_linear)
#A2_psev = BenIsrael(A2_linear[:, 0:(len(A2_linear[0]) - 1)])
X2_estim_linear = solvingSLAE(A2_linear[:, 0:(len(A2_linear[0]) - 1)], np.matrix(A2_linear[:, len(A2_linear[0]) - 1]).T)
print(pd.DataFrame(X2_estim_linear, columns=['X2'], index=['a2', 'c22', 'c23', 'b21']))

           X2
a2   0.040594
c22  0.102694
c23  0.081959
b21  0.000137


###Для функции $x_3[t+1]$

In [78]:
#x3, v1, v3
x3_linear = model_0[['x3(0)', 'v1(0)', 'v3(0)']]
x3_linear['x3(1)'] = model_1['x3(1)']
x3_linear.insert(0, "optional_col", vector_ones)
x3_linear

<ipython-input-78-e8815f496302>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linear['x3(1)'] = model_1['x3(1)']


,optional_col,x3(0),v1(0),v3(0),x3(1)
0,1,0.063997,0.963676,1.171676,0.004210
1,1,-0.876941,11.737576,0.834694,0.345779
2,1,-0.897366,0.207562,0.654753,-0.045032
3,1,0.550901,986.154945,1.245891,-0.913969
4,1,-0.918911,16.872739,0.655107,0.346637
...,...,...,...,...,...
95,1,-0.976701,2.340292,0.066127,0.687843
96,1,0.866777,-0.529938,1.042199,0.007311
97,1,-0.875076,-2.354448,-0.465643,0.032599
98,1,-0.662985,2.201295,0.848201,-0.886988


In [97]:
A3_linear = np.array(x3_linear)
#A3_psev = BenIsrael(A3_linear[:, 0:(len(A3_linear[0]) - 1)])
X3_estim_linear = solvingSLAE(A3_linear[:, 0:(len(A3_linear[0]) - 1)], np.matrix(A3_linear[:, len(A3_linear[0]) - 1]).T)
print(pd.DataFrame(X3_estim_linear, columns=['X3'], index=['a3', 'c33', 'b31', 'b33']))

           X3
a3  -0.232964
c33 -0.154927
b31  0.000229
b33 -0.000363


###Для функции $y_1[t]$

In [79]:
#x1, x2, v1
y1_linear = model_0[['x1(0)', 'x2(0)', 'v1(0)']]
y1_linear['y1(0)'] = model_0['y1(0)']
y1_linear.insert(0, "optional_col", vector_ones)
y1_linear

<ipython-input-79-6e560ed4e872>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linear['y1(0)'] = model_0['y1(0)']


,optional_col,x1(0),x2(0),v1(0),y1(0)
0,1,0.979192,0.693342,0.963676,1.207528
1,1,-0.016469,0.985913,11.737576,-0.000181
2,1,-0.286341,-0.621831,0.207562,-0.936143
3,1,0.358505,-0.539309,986.154945,0.000296
4,1,-0.084503,-0.999441,16.872739,-0.001456
...,...,...,...,...,...
95,1,-0.171005,-0.557088,2.340292,-0.040484
96,1,0.980792,0.882740,-0.529938,-1.950852
97,1,-0.973664,-0.416506,-2.354448,0.632286
98,1,0.941083,0.765912,2.201295,0.473886


In [98]:
A1y_linear = np.array(y1_linear)
#A1y_psev = BenIsrael(A1y_linear[:, 0:(len(A1y_linear[0]) - 1)])
Y1_estim_linear = solvingSLAE(A1y_linear[:, 0:(len(A1y_linear[0]) - 1)], np.matrix(A1y_linear[:, len(A1y_linear[0]) - 1]).T)
print(pd.DataFrame(Y1_estim_linear, columns=['Y1'], index=['a1', 'c11', 'c12', 'b11']))

           Y1
a1  -0.953592
c11 -0.910108
c12 -0.333075
b11  0.000966


###Для функции $y_2[t]$

In [91]:
#v3
y2_linear = model_0[['v3(0)']]
y2_linear['y2(0)'] = model_0['y2(0)']
y2_linear.insert(0, "optional_col", vector_ones)
y2_linear

<ipython-input-91-77de3ab479f4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linear['y2(0)'] = model_0['y2(0)']


,optional_col,v3(0),y2(0)
0,1,1.171676,0.959897
1,1,0.834694,0.860867
2,1,0.654753,0.780361
3,1,1.245891,0.973489
4,1,0.655107,0.780541
...,...,...,...
95,1,0.066127,0.257059
96,1,1.042199,0.929255
97,1,-0.465643,0.670073
98,1,0.848201,0.866079


In [99]:
A2y_linear = np.array(y2_linear)
#A2y_psev = BenIsrael(A2y_linear[:, 0:(len(A2y_linear[0]) - 1)])
Y2_estim_linear = solvingSLAE(A2y_linear[:, 0:(len(A2y_linear[0]) - 1)], np.matrix(A2y_linear[:, len(A2y_linear[0]) - 1]).T)
print(pd.DataFrame(Y2_estim_linear, columns=['Y2'], index=['a2', 'b23']))

           Y2
a2   0.781789
b23 -0.001292


###Для функции $y_3[t]$

In [81]:
#v1, v3
y3_linear = model_0[['v1(0)', 'v3(0)']]
y3_linear['y3(0)'] = model_0['y3(0)']
y3_linear.insert(0, "optional_col", vector_ones)
y3_linear

<ipython-input-81-2a5a15d5e33f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linear['y3(0)'] = model_0['y3(0)']


,optional_col,v1(0),v3(0),y3(0)
0,1,0.963676,1.171676,1.171676
1,1,11.737576,0.834694,0.834694
2,1,0.207562,0.654753,0.654753
3,1,986.154945,1.245891,1.245891
4,1,16.872739,0.655107,0.655107
...,...,...,...,...
95,1,2.340292,0.066127,0.066127
96,1,-0.529938,1.042199,1.042199
97,1,-2.354448,-0.465643,-0.465643
98,1,2.201295,0.848201,0.848201


In [100]:
A3y_linear = np.array(y3_linear)
#A3y_psev = BenIsrael(A3y_linear[:, 0:(len(A3y_linear[0]) - 1)])
Y3_estim_linear = solvingSLAE(A3y_linear[:, 0:(len(A3y_linear[0]) - 1)], np.matrix(A3y_linear[:, len(A3y_linear[0]) - 1]).T)
print(pd.DataFrame(Y3_estim_linear, columns=['Y3'], index=['a3', 'b31', 'b33']))

               Y3
a3   2.328589e-13
b31 -7.068998e-17
b33  1.000000e+00
